In [ ]:
import zipfile
import os

def unzip_document(zip_path, extract_to='documentos'):
    # Create the target directory if it doesn't exist
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Unzip the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Files extracted to {extract_to}")

# Example usage
zip_path = 'Documentos.zip'  # Replace with your zip file path
unzip_document(zip_path)

Files extracted to documentos


In [ ]:
# prompt: do a pip install unicode

!pip install unidecode
!python -m spacy download es_core_news_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# prompt: delete the fder documentos_procesados

import shutil

def delete_directory(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"Directory '{dir_path}' deleted successfully.")
    except FileNotFoundError:
        print(f"Directory '{dir_path}' not found.")
    except OSError as e:
        print(f"Error deleting directory '{dir_path}': {e}")

# Example usage
delete_directory('documentos_procesados')

Directory 'documentos_procesados' not found.


In [ ]:
import os
import re
import nltk
import spacy
from nltk.corpus import stopwords
from unidecode import unidecode

# Download necessary NLTK resources
nltk.download('stopwords')

# Initialize spaCy with Spanish model
nlp = spacy.load("es_core_news_lg")

# Initialize Spanish stopwords
stop_words = set(stopwords.words('spanish'))

# Define source and destination folders
carpeta_origen = 'documentos'
carpeta_destino = 'documentos_procesados'

# Create destination folder if it doesn't exist
os.makedirs(carpeta_destino, exist_ok=True)

def preprocesar_nombre_archivo(nombre_archivo):
    # Split the filename by '_'
    partes = nombre_archivo.split('_')

    # Process each part
    partes_procesadas = []
    for parte in partes:
        # Remove non-alphanumeric characters
        parte = re.sub(r'\W+', ' ', parte)

        # Convert to lowercase and remove accents
        parte = unidecode(parte.lower())

        # Process with spaCy
        doc = nlp(parte)

        # Filter stopwords and get lemmas
        palabras = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]

        # Join processed words for this part
        parte_procesado = '_'.join(palabras)

        if parte_procesado:
            partes_procesadas.append(parte_procesado)

    # Join all processed parts
    nombre_procesado = '_'.join(partes_procesadas)

    return nombre_procesado

def preprocesar_texto(texto):
    # Convert to lowercase and remove accents
    texto = unidecode(texto.lower())

    # Remove non-alphanumeric characters
    texto = re.sub(r'\W+', ' ', texto)

    # Process with spaCy
    doc = nlp(texto)

    # Filter words and get lemmas
    palabras = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]

    # Join processed words
    texto_procesado = ' '.join(palabras)

    return texto_procesado

def procesar_archivos(carpeta_origen, carpeta_destino):
    for archivo in os.listdir(carpeta_origen):
        if archivo.endswith('.txt'):
            ruta_origen = os.path.join(carpeta_origen, archivo)

            # Preprocess filename
            nombre_procesado = preprocesar_nombre_archivo(archivo)
            ruta_destino = os.path.join(carpeta_destino, f"{archivo}.txt")

            try:
                # Read original file
                with open(ruta_origen, 'r', encoding='utf-8') as f:
                    contenido = f.read()

                # Process file content
                contenido_procesado = preprocesar_texto(contenido)

                # Save processed content with processed filename
                with open(ruta_destino, 'w', encoding='utf-8') as f:
                    f.write(contenido_procesado)

                print(f"Processed: {archivo} -> {nombre_procesado}.txt")

            except Exception as e:
                print(f"Error processing {archivo}: {e}")

if __name__ == "__main__":
    procesar_archivos(carpeta_origen, carpeta_destino)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Processed: 68_Enologia_y_Cocteleria.txt -> enologio_cocteleria_txt.txt
Processed: 186_Sistemas_Lean.txt -> sistema_lean_txt.txt
Processed: 56_Carniceria.txt -> carniceria_txt.txt
Processed: 6_Ingles_Nivel_1.txt -> ingl_nivel_txt.txt
Processed: 113_Negocios_Internacionales.txt -> negocio_internacional_txt.txt
Processed: 66_Reposteria_y_Chocolateria.txt -> reposteria_chocolaterio_txt.txt
Processed: 133_Taller_de_Arte_2.txt -> taller_arte_txt.txt
Processed: 46_Matematica_Empresarial_+Ej.txt -> matematico_empresarial_ej_txt.txt
Processed: 100_Principios_de_Seguros.txt -> principio_seguro_txt.txt
Processed: 82_Metodologias_de_Ensenanza.txt -> metodologia_ensenanza_txt.txt
Processed: 130_Arte_y_Contexto_Social.txt -> arte_contexto_social_txt.txt
Processed: 13_Ingles_Nivel_4.txt -> ingl_nivel_txt.txt
Processed: 84_Ensenanza_de_Ciencias_Sociales.txt -> ensenanza_ciencia_social_txt.txt
Processed: 7_Ingles_Nivel_2.txt -> ingl_nivel_txt.txt
Processed: 97_Principios_de_Marketing.txt -> principio_m

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib  # Para guardar el vectorizador

def calculate_tf_idf(folder_path, output_file, vectorizer_file):
    """
    Calculates TF-IDF vectors for documents in a folder, saves them to a CSV file,
    and saves the TF-IDF vectorizer as a .joblib file.
    """

    # Initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Store filenames and text contents
    filenames = []
    documents = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(folder_path, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as file:
                    text = file.read()
                    filenames.append(filename)
                    documents.append(text)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    # Fit and transform the documents
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Save the TF-IDF vectorizer to a .joblib file
    joblib.dump(vectorizer, vectorizer_file)
    print(f"TF-IDF vectorizer saved to {vectorizer_file}")

    # Create a DataFrame from the TF-IDF matrix
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    # Insert filenames as the first column
    tfidf_df.insert(0, 'filename', filenames)

    # Save the DataFrame to a CSV file
    tfidf_df.to_csv(output_file, index=False)
    print(f"TF-IDF vectors saved to {output_file}")

# Example usage
folder_path = 'documentos_procesados'
output_file = 'tf_idf_documentos.csv'
vectorizer_file = 'tfidf_vectorizer.joblib'
calculate_tf_idf(folder_path, output_file, vectorizer_file)


TF-IDF vectorizer saved to tfidf_vectorizer.joblib
TF-IDF vectors saved to tf_idf_documentos.csv


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
import joblib  # Para guardar el vectorizador

def calculate_log_tf_idf(folder_path, output_file, vectorizer_file):
    """
    Calculates logarithmic TF and standard IDF vectors for documents in a folder,
    and saves the CountVectorizer as a .joblib file.
    """
    # Store filenames and text contents
    filenames = []
    documents = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(folder_path, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as file:
                    text = file.read()
                    filenames.append(filename)
                    documents.append(text)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    # Create count vectorizer
    count_vectorizer = CountVectorizer()
    term_freq_matrix = count_vectorizer.fit_transform(documents)

    # Save the CountVectorizer to a .joblib file
    joblib.dump(count_vectorizer, vectorizer_file)
    print(f"CountVectorizer saved to {vectorizer_file}")

    # Get feature names
    feature_names = count_vectorizer.get_feature_names_out()

    # Calculate logarithmic TF
    log_tf_matrix = np.log1p(term_freq_matrix.toarray())

    # Calculate IDF
    doc_count = len(documents)
    idf_vector = np.log(doc_count / (np.sum(term_freq_matrix.toarray() > 0, axis=0) + 1))

    # Calculate TF-IDF
    tfidf_matrix = log_tf_matrix * idf_vector

    # Create DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix, columns=feature_names)
    tfidf_df.insert(0, 'filename', filenames)

    # Save to CSV
    tfidf_df.to_csv(output_file, index=False)
    print(f"Logarithmic TF-IDF vectors saved to {output_file}")

# Example usage
folder_path = 'documentos_procesados'
output_file = 'tf_idf_documentos_2.csv'
vectorizer_file = 'count_vectorizer.joblib'
calculate_log_tf_idf(folder_path, output_file, vectorizer_file)


CountVectorizer saved to count_vectorizer.joblib
Logarithmic TF-IDF vectors saved to tf_idf_documentos_2.csv


In [ ]:
queries = [
    "curso de programación, diseño gráfico y cocina internacional con técnicas avanzadas de cocina y recetas modernas",
    "Curso de escritura creativa y técnica para estudiantes de música interesados en mejorar sus habilidades de composición",
    "curso de idiomas inicial para estudiantes que quieren aprender inglés de manera interactiva y efectiva con enfoque en conversación",
    "curso sobre inteligencia artificial, tecnología de vanguardia, innovación disruptiva y desarrollo de aplicaciones inteligentes en el mercado",
    "curso para estudios de matemáticas aplicadas y artes liberales, combinando teoría matemática avanzada con análisis crítico cultural",
]

queries_procesadas = []

for query in queries:
    # Convert to lowercase and remove accents
    query_pre = unidecode(query.lower())

    # Remove non-alphanumeric characters
    query_pre = re.sub(r'\W+', ' ', query_pre)

    # Process with spaCy
    doc = nlp(query_pre)

    # Filter words and get lemmas
    palabras = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]

    # Join processed words
    query_preprocesada = ' '.join(palabras)

    queries_procesadas.append(query_preprocesada)

for query in queries_procesadas:
    print(query)


curso programacion diseno grafico cocina internacional tecnica avanzado cocina receta moderno
curso escritura creativo tecnico estudiante musica interesado mejorar habilidad composicion
curso idioma inicial estudiante querer aprender ingl manera interactivo efectivo enfoque conversacion
curso inteligencia artificial tecnologia vanguardia innovacion disruptivo desarrollo aplicación inteligente mercado
curso estudio matematica aplicado art liberal combinar teoria matematico avanzado analisis critico cultural


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar el vectorizador entrenado (asegúrate de haberlo guardado previamente)
import joblib
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

queries_vector = []

for query in queries_procesadas:
    query_vector = tfidf_vectorizer.transform([query])
    print(query_vector)
    queries_vector.append(query_vector)

  (0, 107)	0.20077436589628997
  (0, 180)	0.5439117445998276
  (0, 310)	0.08069243018616552
  (0, 371)	0.24995123897947033
  (0, 589)	0.3641364022100168
  (0, 688)	0.3437261557832206
  (0, 790)	0.2945492827000402
  (0, 939)	0.2787178656358544
  (0, 976)	0.3929030288664009
  (0, 1087)	0.1590725296477077
  (0, 214)	0.384278316914726
  (0, 290)	0.42675196410684546
  (0, 310)	0.09456800490558967
  (0, 453)	0.42675196410684546
  (0, 486)	0.10147876478095012
  (0, 599)	0.2201392895696019
  (0, 686)	0.46046519451125834
  (0, 773)	0.29866849267548295
  (0, 1088)	0.34519889842270446
  (0, 64)	0.29758942365223656
  (0, 268)	0.45719752518264517
  (0, 310)	0.10131472480691577
  (0, 394)	0.3593342657866509
  (0, 426)	0.29758942365223656
  (0, 486)	0.10871851571566776
  (0, 660)	0.4116937009123579
  (0, 681)	0.39545268304823084
  (0, 751)	0.36982625817539655
  (0, 59)	0.2494217585375952
  (0, 81)	0.4524281995392851
  (0, 310)	0.09291740766394044
  (0, 339)	0.26431666429423034
  (0, 664)	0.3775710935

In [ ]:
print(queries_vector[1])

  (0, 214)	0.384278316914726
  (0, 290)	0.42675196410684546
  (0, 310)	0.09456800490558967
  (0, 453)	0.42675196410684546
  (0, 486)	0.10147876478095012
  (0, 599)	0.2201392895696019
  (0, 686)	0.46046519451125834
  (0, 773)	0.29866849267548295
  (0, 1088)	0.34519889842270446


In [ ]:
# prompt: save queries_vector as a csv file, the first column needs to be the index of the vector

import pandas as pd
import numpy as np

# Assuming queries_vector is already defined as in your provided code

# Create an empty list to store the data for the CSV file
data_for_csv = []

# Iterate through the queries_vector and their corresponding indices
for i, query_vector in enumerate(queries_vector):
    # Convert the sparse matrix to a dense array
    dense_vector = query_vector.toarray()[0]

    # Create a row for the CSV, with the index as the first element
    row = [i] + list(dense_vector)
    data_for_csv.append(row)

# Create column names
column_names = ['index'] + [f'feature_{i}' for i in range(len(dense_vector))]

# Create a pandas DataFrame from the data
df = pd.DataFrame(data_for_csv, columns=column_names)

# Save the DataFrame to a CSV file
df.to_csv('queries_vector.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar el vectorizador entrenado (asegúrate de haberlo guardado previamente)
import joblib
tfidf_vectorizer = joblib.load('count_vectorizer.joblib')

queries_vector_2 = []

for query in queries_procesadas:
    query_vector = tfidf_vectorizer.transform([query])
    print(query_vector)
    queries_vector_2.append(query_vector)

  (0, 107)	1
  (0, 180)	2
  (0, 310)	1
  (0, 371)	1
  (0, 589)	1
  (0, 688)	1
  (0, 790)	1
  (0, 939)	1
  (0, 976)	1
  (0, 1087)	1
  (0, 214)	1
  (0, 290)	1
  (0, 310)	1
  (0, 453)	1
  (0, 486)	1
  (0, 599)	1
  (0, 686)	1
  (0, 773)	1
  (0, 1088)	1
  (0, 64)	1
  (0, 268)	1
  (0, 310)	1
  (0, 394)	1
  (0, 426)	1
  (0, 486)	1
  (0, 660)	1
  (0, 681)	1
  (0, 751)	1
  (0, 59)	1
  (0, 81)	1
  (0, 310)	1
  (0, 339)	1
  (0, 664)	1
  (0, 676)	1
  (0, 777)	1
  (0, 1089)	1
  (0, 49)	1
  (0, 60)	1
  (0, 78)	1
  (0, 107)	1
  (0, 197)	1
  (0, 296)	1
  (0, 308)	1
  (0, 310)	1
  (0, 488)	1
  (0, 757)	1
  (0, 758)	1
  (0, 1097)	1


In [ ]:
# prompt: save queries_vector as a csv file, the first column needs to be the index of the vector

import pandas as pd
import numpy as np

# Assuming queries_vector is already defined as in your provided code

# Create an empty list to store the data for the CSV file
data_for_csv = []

# Iterate through the queries_vector and their corresponding indices
for i, query_vector in enumerate(queries_vector_2):
    # Convert the sparse matrix to a dense array
    dense_vector = query_vector.toarray()[0]

    # Create a row for the CSV, with the index as the first element
    row = [i] + list(dense_vector)
    data_for_csv.append(row)

# Create column names
column_names = ['index'] + [f'feature_{i}' for i in range(len(dense_vector))]

# Create a pandas DataFrame from the data
df = pd.DataFrame(data_for_csv, columns=column_names)

# Save the DataFrame to a CSV file
df.to_csv('queries_vector_2.csv', index=False)